<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 0.Imports

In [3]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")
print('finished')

finished


## <span style='color:#ff5f27'> 🌍 1.1 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [5]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
with open('../../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
#Now we just need to modify the first nb. The other ones are automatically updated
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

print(location_str)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1150100
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
{"country": "italy", "city": "toscana", "street": "pisa", "aqicn_url": "https://api.waqi.info/feed/@9432", "latitude": 43.47, "longitude": 10.68}


### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [ ]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
weather_fg

---

## <span style='color:#ff5f27'> 2.Retrieve Datas using APIs</span>


### <span style='color:#ff5f27'> 2.1 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [ ]:
#import requests
#import pandas as pd
#yesterday = today - datetime.timedelta(days=3)

#aq_tmp_df = util.get_pm25(aqicn_url, country, city, street, yesterday, AQI_API_KEY)
# Function that gives back df from API request 
#aq_tmp_df

In [ ]:
#fin_df = pd.concat([fin_df,aq_tmp_df])

#fin_df

In [ ]:
#air_quality_fg.insert(existing_df)

In [ ]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

In [ ]:
#aq_today_df.info()

In [ ]:
dfa = air_quality_fg.read()
dfa = dfa.sort_values(by='date', ascending=False)  # Sorts in ascending order
dfa

### <span style='color:#ff5f27'> 2.2 Get Weather Forecast data</span>

In [ ]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
# Function that gives back df from API request 
hourly_df.head()

In [ ]:
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df.head()

In [ ]:
daily_df.info()

## <span style="color:#ff5f27;">3.3 ⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Insert new data
air_quality_fg.insert(aq_today_df)

In [ ]:
# Insert new data
air_quality_fg.insert(aq_today_df)

In [ ]:
# Insert new data
weather_fg.insert(daily_df)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store


In [ ]:
dfa['pm25_rolling'] = dfa['pm25'].rolling(window=3, min_periods=1).mean()
dfa['pm25_rolling'] = dfa['pm25_rolling'].fillna(dfa['pm25'])
dfa

## Adding the Rolling Windows Datagroup

In [ ]:
air_quality_roll_fg = fs.get_or_create_feature_group(
    name='air_quality_roll',
    description='Air Quality characteristics of each day, rolling feature',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date"
)

In [ ]:
air_quality_roll_fg.insert(dfa)

In [ ]:
df_lagged = air_quality_roll_fg.read()

In [ ]:
df_lagged